In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
import csv

In [9]:
#file_name = '\sample2.tsv'
file_name = r'\task_a_distant.tsv'
#df = pd.read_table(r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name)
df = pd.read_table(r'C:\Users\Mukuu\Desktop\MSc Proj\English' + file_name)
df

,id,text,average,std
0,1159533701283352576,"First time I heard his name in camp, he seems ...",0.195773,0.187379
1,1159533703522992128,When I go to drink with Tsubaki he would alway...,0.262401,0.145998
2,1159533703758061570,@USER His ass need to stay up 😂😂,0.833391,0.140628
3,1159533703904800769,most important tweet of the day : Fuck Donald ...,0.565238,0.187498
4,1159533705125343232,You wanna leave? then feel free cus I promise ...,0.664921,0.097098
...,...,...,...,...
9075413,1187517463640137728,Why everyone should vote for MAMAMOO as The Be...,0.174259,0.205074
9075414,1187636341338710016,@USER Yes but Boris is a puppet for the ERG an...,0.487383,0.067990
9075415,1187636341560889344,@USER Did you check or just messaing without s...,0.241537,0.176056
9075416,1187636341661679617,I’ll never forget the day I dropped my casava ...,0.251834,0.170168


In [10]:
file_name[:-4]

'\\task_a_distant'

In [11]:
df.shape

(9075418, 4)

In [12]:
#decontract
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [13]:
sentence = []
for i in df['text']:
    sentence.append(i)
#sentence

# Text Cleanning

In [6]:
def clean_text(sentence):
    trans = str.maketrans('','', string.punctuation)
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)
    stop_words = set(stopwords.words('english'))
    for i in range(len(sentence)):
        sentence[i] = sentence[i].lower()                # To lower case
        sentence[i] = re.sub(r'\d+', '', sentence[i])    # Remove numbers
        sentence[i] = re.sub(r'’',r"'",sentence[i])     # Modify astrophies
        sentence[i] = re.sub(r'”',r"'",sentence[i])
        sentence[i] = re.sub('@user', '', sentence[i])   # Remove @user
        sentence[i] = sentence[i].translate(trans)       # Remove punctuations
        sentence[i] = emoji_pattern.sub(r'', sentence[i])# Remove imojis
        #sentence[i] = sentence[i].strip()                # Remove white space
        tokens = word_tokenize(sentence[i])
        #print(tokens)
        result = [i for i in tokens if not i in stop_words]
        #print(result)
        sentence[i] = result
    return sentence

In [15]:
clean_tokens = clean_text(sentence)

In [16]:
#clean_tokens

In [17]:
#len(clean_tokens)

# Concatenate tokens back to sentences

In [8]:
# Concatenating everything
def Concat(list_tokens):
    result = []
    for i in list_tokens:
        temp = ''
        for j in i:
            temp = temp + j + ' '
        #print(temp)
        result.append(temp[:-1])
    return result
#clean_sentences = Concat(clean_tokens)
#clean_sentences

# Create new csv file with cleaned text

In [19]:
labels = []
for i in df['average']:
    if i >= 0.5:
        labels.append('OFF')
    else:
        labels.append('NOT')
#labels

In [20]:
cleaned = {'id': df['id'],
           'text' : clean_sentences,
           'label' : labels}
df_clean = pd.DataFrame(cleaned, columns = ['id','text','label'])

In [21]:
df_clean

,id,text,label
0,1159533701283352576,first time heard name camp seems forgotten guy...,NOT
1,1159533703522992128,go drink tsubaki would always fall asleep firs...,NOT
2,1159533703758061570,ass need stay,OFF
3,1159533703904800769,important tweet day fuck donald trump whole pa...,OFF
4,1159533705125343232,wan na leave feel free cus promise wont stop y...,OFF
...,...,...,...
9075413,1187517463640137728,everyone vote mamamoo best female group powerf...,NOT
9075414,1187636341338710016,yes boris puppet erg billionaires behind misse...,NOT
9075415,1187636341560889344,check messaing without seeing already done sop...,NOT
9075416,1187636341661679617,ill never forget day dropped casava leaves off...,NOT


In [22]:
file_name[:-4]

'\\task_a_distant'

In [23]:
df_clean.to_csv (r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name[:-4] + '_cleaned.csv', index = False, header=True)

In [24]:
df_clean['label'].value_counts()

NOT    7628650
OFF    1446768
Name: label, dtype: int64

In [27]:
# OLID testset task A
#file_name = '\sample2.tsv'
file_name = r'\testset-levela_uncleaned.csv'
#df = pd.read_table(r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name)
df = pd.read_csv(r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name)
df

,id,text,label
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
1,27014,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,13876,#Watching #Boomer getting the news that she is...,NOT
4,60133,#NoPasaran: Unity demo to oppose the far-right...,OFF
...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,OFF
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,NOT
857,67018,3 people just unfollowed me for talking about ...,OFF
858,50665,#WednesdayWisdom Antifa calls the right fascis...,NOT


In [28]:
sentence = []
for i in df['text']:
    sentence.append(i)
sentence

['#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon \u2066@USER URL',
 '#ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.',
 '#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendment #RNC #USMC #veterans @USER    @USER @USER @USER @USER   #fakereporting #THESWAMP #dnc #liberals @USER @USER #fakeoutrage @USER  First, it reduces the ca URL',
 '#Watching #Boomer getting the news that she is still up for parole always makes me smile. #Wentworth Finale...@USER is such a treasure. URL',
 '#NoPasaran: Unity demo to oppose the far-right in #London – #antifa #Oct13 — Enough is Enough! URL',
 '. . . What the fuck did he do this time?',
 '#RAP is a form of ART! Used to express yourself freely. It does not gv the green light or excuse the behavior of acting like a

In [29]:
clean_tokens = clean_text(sentence)
clean_sentences = Concat(clean_tokens)
cleaned = {'id': df['id'],
           'text' : clean_sentences,
           'label' : df['label']}
df_clean = pd.DataFrame(cleaned, columns = ['id','text','label'])

In [30]:
df_clean

,id,text,label
0,15923,whoisq wherestheserver dumpnike declasfisa dem...,OFF
1,27014,constitutionday revered conservatives hated pr...,NOT
2,30530,foxnews nra maga potus trump ndamendment rnc u...,NOT
3,13876,watching boomer getting news still parole alwa...,NOT
4,60133,nopasaran unity demo oppose farright london – ...,OFF
...,...,...,...
855,73439,despicabledems lie rifles dem distorted law pu...,OFF
856,25657,meetthespeakers present event oiw finpact glob...,NOT
857,67018,people unfollowed talking merlin sorry yall im...,OFF
858,50665,wednesdaywisdom antifa calls right fascist rea...,NOT


In [32]:
df_clean.to_csv (r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name[:-14] + '_cleaned.csv', index = False, header=True)

In [9]:
# SOLID testset task A B and C
#file_name = '\sample2.tsv'
file_name = r'\test_b_tweets_uncleaned.csv'
#df = pd.read_table(r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name)
df = pd.read_csv(r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name)
df

,id,text,label
0,BC0,Then to top it completely the fuck off my trai...,UNT
1,BC1,"To which, of course, many Slovenes will respon...",TIN
2,BC2,@USER it means go away with your sorry ass kkkkk,TIN
3,BC3,@USER @USER And he’s closer to trump’s ass. Gr...,TIN
4,BC4,Looking back at old photos of me makes me phys...,UNT
...,...,...,...
1417,BC1417,@USER His name is cheating ass Myron 💀💀💀💀💀,TIN
1418,BC1418,"@USER So do I.....my neighbours, the Postie an...",UNT
1419,BC1419,@USER @USER That shit is badass,UNT
1420,BC1420,I get to work hop on here co workers around an...,UNT


In [10]:
sentence = []
for i in df['text']:
    sentence.append(i)
sentence

['Then to top it completely the fuck off my trainer was my boyfriend. And he went the fuck hard on me. So of course everything was tight',
 'To which, of course, many Slovenes will respond along the lines of: "You fool, Slovenes are the worst!"',
 '@USER it means go away with your sorry ass kkkkk',
 '@USER @USER And he’s closer to trump’s ass. Greatly shortens the kiss time',
 "Looking back at old photos of me makes me physically sick like why was I allowed to be fugly as hell?? Who let me do that y'all fake as fuck",
 "I used to believe you weren't a shit of a person, but now I'm rethinking that maybe.. you are.",
 'Trying to act like u don’t give a fuck when u give a fuck is hard like',
 '@USER i even have them blocked but they seem to always find their way back to accuse me of shit i didnt do',
 'miley getting dragged over the most trivial shit: a thread 💫',
 'On my way to this nut ass job 😩',
 'fuck a relationship, stay single and get ya money up',
 'Even Douglas knows his daddy is

In [11]:
clean_tokens = clean_text(sentence)
clean_sentences = Concat(clean_tokens)
cleaned = {'id': df['id'],
           'text' : clean_sentences,
           'label' : df['label']}
df_clean = pd.DataFrame(cleaned, columns = ['id','text','label'])

In [12]:
df_clean.to_csv (r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name[:-14] + '_cleaned.csv', index = False, header=True)

In [40]:
# SOLID train task B
#file_name = '\sample2.tsv'
file_name = r'\task_b_distant.tsv'
#df = pd.read_table(r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name)
df = pd.read_table(r'C:\Users\Mukuu\Desktop\MSc Proj\English' + file_name)
df

,id,text,average,std
0,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.230133,0.219593
1,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.269860,0.178943
2,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.229661,0.228530
3,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.169093,0.180201
4,1159533742366633984,@USER @USER Fuck him better than his hoes,0.274980,0.276721
...,...,...,...,...
188969,1187532615030587394,"If you lying bout lil shit, you lying bout a l...",0.231188,0.214216
188970,1187611970624610304,do y’all ever like wanna grab somebodys face a...,0.237955,0.219919
188971,1187636295306268673,I meant tool but the point is women ain’t shit...,0.594655,0.243256
188972,1187636299785744384,@USER i sang too the fuck-,0.546231,0.310368


In [45]:
sentence = []
for i in df['text']:
    sentence.append(i)
sentence

['@USER Trump is a fucking idiot his dementia is getting worse',
 '@USER HELL YES! His grinned and thumbs up are disgusting!',
 "@USER Can't wait to see the shit show his death will bring.",
 '@USER @USER @USER This guys is dumb check his latest tweets he is sick',
 '@USER @USER Fuck him better than his hoes',
 'Junhee and I are gonna take turns beating his sleep paralysis ghost’s ass!!!!¡ step the fuck up demon lady !!!!!!',
 '@USER Exactly, a pervert.  So disgusting and he has women in his house.',
 'Invest in your bitch',
 'everyone talks shit in LA',
 '@USER Who the fuck are you to discuss interfere in our personal matters',
 'Yeah ngl lion king had me in fucking pieces 😂',
 "@USER Ffs, We have our best window in forever and you're still on a fucking downer. Fuck off already 😅",
 'i support everyone in this world 🙏 except you donald trump supporters, fuck y’all and eat shit',
 '@USER these bitches want dick in they mouf',
 '@USER Innit tho☹ I mean everyone talks shit everywhere but

In [46]:
labels = []
for i in df['average']:
    if i >= 0.5:
        labels.append('UNT')
    else:
        labels.append('TIN')

In [47]:
clean_tokens = clean_text(sentence)
clean_sentences = Concat(clean_tokens)
cleaned = {'id': df['id'],
           'text' : clean_sentences,
           'label' : labels}
df_clean = pd.DataFrame(cleaned, columns = ['id','text','label'])

In [48]:
df_clean.to_csv (r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name[:-4] + '_cleaned.csv', index = False, header=True)

In [49]:
df_clean['label'].value_counts()

TIN    149550
UNT     39424
Name: label, dtype: int64

In [58]:
# SOLID train task B
#file_name = '\sample2.tsv'
file_name = r'\task_c_distant_ann.tsv'
#df = pd.read_table(r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name)
df = pd.read_table(r'C:\Users\Mukuu\Desktop\MSc Proj\English' + file_name)
df

,id,text,average_ind,average_grp,average_oth,std_ind,std_grp,std_oth
0,1159533712079503361,@USER Trump is a fucking idiot his dementia is...,0.833432,0.076110,0.107765,0.208334,0.098937,0.138649
1,1159533713044234241,@USER HELL YES! His grinned and thumbs up are ...,0.481062,0.367363,0.138841,0.345225,0.335924,0.083230
2,1159533718345830400,@USER Can't wait to see the shit show his deat...,0.438813,0.268574,0.377573,0.182609,0.186880,0.254621
3,1159533739871002625,@USER @USER @USER This guys is dumb check his ...,0.712995,0.123504,0.111130,0.248839,0.107572,0.067552
4,1159533742366633984,@USER @USER Fuck him better than his hoes,0.691414,0.146723,0.192282,0.204415,0.154818,0.104436
...,...,...,...,...,...,...,...,...
188968,1187532615030587394,"If you lying bout lil shit, you lying bout a l...",0.690031,0.178021,0.201831,0.249506,0.189998,0.166256
188969,1187611970624610304,do y’all ever like wanna grab somebodys face a...,0.503566,0.316713,0.260821,0.302630,0.146222,0.220385
188970,1187636295306268673,I meant tool but the point is women ain’t shit...,0.363758,0.431517,0.282849,0.239436,0.165351,0.095065
188971,1187636299785744384,@USER i sang too the fuck-,0.598230,0.161560,0.251305,0.191871,0.107075,0.115206


In [59]:
sentence = []
for i in df['text']:
    sentence.append(i)
sentence

['@USER Trump is a fucking idiot his dementia is getting worse',
 '@USER HELL YES! His grinned and thumbs up are disgusting!',
 "@USER Can't wait to see the shit show his death will bring.",
 '@USER @USER @USER This guys is dumb check his latest tweets he is sick',
 '@USER @USER Fuck him better than his hoes',
 'Junhee and I are gonna take turns beating his sleep paralysis ghost’s ass!!!!¡ step the fuck up demon lady !!!!!!',
 '@USER Exactly, a pervert.  So disgusting and he has women in his house.',
 'Invest in your bitch',
 'everyone talks shit in LA',
 '@USER Who the fuck are you to discuss interfere in our personal matters',
 'Yeah ngl lion king had me in fucking pieces 😂',
 "@USER Ffs, We have our best window in forever and you're still on a fucking downer. Fuck off already 😅",
 'i support everyone in this world 🙏 except you donald trump supporters, fuck y’all and eat shit',
 '@USER these bitches want dick in they mouf',
 '@USER Innit tho☹ I mean everyone talks shit everywhere but

In [62]:
labels = []
for i in range(len(df['average_ind'])):
    if df['average_ind'][i] > df['average_grp'][i] and df['average_ind'][i] > df['average_oth'][i]:
        labels.append('IND')
    elif df['average_grp'][i] > df['average_ind'][i] and df['average_grp'][i] > df['average_oth'][i]:
        labels.append('GRP')
    else:
        labels.append('OTH')

In [63]:
clean_tokens = clean_text(sentence)
clean_sentences = Concat(clean_tokens)
cleaned = {'id': df['id'],
           'text' : clean_sentences,
           'label' : labels}
df_clean = pd.DataFrame(cleaned, columns = ['id','text','label'])

In [64]:
df_clean.to_csv (r'C:\Users\Mukuu\Desktop\MSc Proj' + file_name[:-4] + '_cleaned.csv', index = False, header=True)

In [65]:
df_clean['label'].value_counts()

IND    152562
GRP     24917
OTH     11494
Name: label, dtype: int64

In [66]:
df_clean['label'].value_counts()

IND    152562
GRP     24917
OTH     11494
Name: label, dtype: int64